In [ ]:
# Standard library
import json
import os
import random
import time

# Third-party libraries
import joblib
import matplotlib.pyplot as plt
import numpy as np
import optuna
import pandas as pd
import torch

# Scikit-learn - core modules
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, OneHotEncoder, StandardScaler
from sklearn.utils.class_weight import compute_class_weight

# Scikit-learn - metrics
from sklearn.metrics import (
    accuracy_score, average_precision_score, balanced_accuracy_score,
    ConfusionMatrixDisplay, f1_score, log_loss,
    matthews_corrcoef, mean_squared_error, precision_score,
    PrecisionRecallDisplay, r2_score, recall_score, roc_auc_score, RocCurveDisplay
)

# Local application/library imports
from utils import load_search_space, get_model


## DATASET

In [ ]:
SEED = 64

# Set random seeds
torch.manual_seed(SEED)
random.seed(SEED)
np.random.seed(SEED)

In [ ]:
# Dataset Info
# adult_income_cleaned, framingham_cleaned, preprocessed_heloc, diabetes
dataset_name = 'mfeat-fourier'        
dataset_subpath = 'Multiclass/mfeat-fourier'       
task_type = 'Multiclass'

In [ ]:
# Dataset Info
# adult_income_cleaned, framingham_cleaned, preprocessed_heloc, diabetes
dataset_name = 'boston'        
dataset_subpath = 'Regression/boston'       
task_type = 'Regression'

In [ ]:
df = pd.read_csv(f"./data/{dataset_subpath}/{dataset_name}.csv")

In [ ]:
df.shape

In [ ]:
df.head()

## LOAD AND PREPROCESS

In [ ]:
def preprocess_data(df, dataset_name, task_type, model_type="default", seed=42):
    task_type = task_type.lower()
    model_type = model_type.lower()

    # Load config
    with open(f"./configs/preprocess/{dataset_name}.json") as f:
        config = json.load(f)

    categorical_cols = config["categorical_cols"]
    numerical_cols = config["numerical_cols"]
    encoding = config["encoding"]

    # Extract features and target
    X = df[numerical_cols + categorical_cols].copy()
    y = df.iloc[:, -1].copy()

    # Encode target if needed
    le = None
    if encoding.get("target") == "label":
        le = LabelEncoder()
        y = le.fit_transform(y)
        label_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
    else:
        label_mapping = None

    # Split raw data before transformation
    if task_type == "regression":
        # For regression, we can use a simple split
        X_train_raw, X_temp_raw, y_train, y_temp = train_test_split(
            X, y, test_size=0.3, random_state=seed
        )
        X_val_raw, X_test_raw, y_val, y_test = train_test_split(
            X_temp_raw, y_temp, test_size=0.5, random_state=seed
        )
    else:
        # For classification, we need stratified splits
        X_train_raw, X_temp_raw, y_train, y_temp = train_test_split(
            X, y, test_size=0.3, random_state=seed, stratify=y
        )
        X_val_raw, X_test_raw, y_val, y_test = train_test_split(
            X_temp_raw, y_temp, test_size=0.5, random_state=seed, stratify=y_temp
        )

    # Ensure y_* are Series with index matching the X_*
    y_train = pd.Series(y_train, index=X_train_raw.index)
    y_val = pd.Series(y_val, index=X_val_raw.index)
    y_test = pd.Series(y_test, index=X_test_raw.index)

    # Compute class weights for classification
    class_weight = None
    if task_type in ["binary", "multiclass"]:
        class_weight_values = compute_class_weight("balanced", classes=np.unique(y_train), y=y_train)
        # Create the class weight dictionary with keys as native Python int (not numpy.int32)
        class_weight = dict(zip([int(key) for key in np.unique(y_train)], class_weight_values))
        print(f"Class weights: {class_weight}")

    # CATBOOST path (no transformation, native categorical handling)
    if model_type == "catboost":
        for col in categorical_cols:
            X_train_raw[col] = X_train_raw[col].astype(str)
            X_val_raw[col] = X_val_raw[col].astype(str)
            X_test_raw[col] = X_test_raw[col].astype(str)
        print(f"Shapes — Train: {X_train_raw.shape}, Val: {X_val_raw.shape}, Test: {X_test_raw.shape}")
        print(f"Numerical features: {len(numerical_cols)} — {numerical_cols}")
        print(f"Categorical features: {len(categorical_cols)} — {categorical_cols}")
        print(f"Total features: {X_train_raw.shape[1]}")
        if label_mapping:
            print(f"Target label mapping: {label_mapping}")
        return (
            X_train_raw, X_val_raw, X_test_raw,
            y_train, y_val, y_test,
            categorical_cols, le, class_weight
        )

    # Transform numerical and categorical features
    transformers = []

    if encoding["numerical_features"] == "minmax":
        transformers.append(("num", MinMaxScaler(), numerical_cols))
    elif encoding["numerical_features"] == "standard":
        transformers.append(("num", StandardScaler(), numerical_cols))

    if categorical_cols and encoding["categorical_features"] == "onehot":
        transformers.append(("cat", OneHotEncoder(sparse_output=False, handle_unknown="ignore"), categorical_cols))

    if transformers:
        preprocessor = ColumnTransformer(transformers=transformers)
        X_train = preprocessor.fit_transform(X_train_raw)
        X_val = preprocessor.transform(X_val_raw)
        X_test = preprocessor.transform(X_test_raw)

        # Recover transformed column names
        if "cat" in preprocessor.named_transformers_:
            cat_feature_names = preprocessor.named_transformers_["cat"].get_feature_names_out(categorical_cols)
            all_feature_names = numerical_cols + list(cat_feature_names)
        else:
            all_feature_names = numerical_cols + categorical_cols

        X_train = pd.DataFrame(X_train, columns=all_feature_names, index=X_train_raw.index)
        X_val = pd.DataFrame(X_val, columns=all_feature_names, index=X_val_raw.index)
        X_test = pd.DataFrame(X_test, columns=all_feature_names, index=X_test_raw.index)
    else:
        all_feature_names = numerical_cols + categorical_cols  # or keep original order
        X_train = pd.DataFrame(X_train_raw, columns=all_feature_names, index=X_train_raw.index)
        X_val = pd.DataFrame(X_val_raw, columns=all_feature_names, index=X_val_raw.index)
        X_test = pd.DataFrame(X_test_raw, columns=all_feature_names, index=X_test_raw.index)

    print(f"Shapes — Train: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}")
    print(f"Numerical features: {len(numerical_cols)} — {numerical_cols}")
    print(f"Categorical features: {len(categorical_cols)} — {categorical_cols}")
    print(f"Total features: {X_train.shape[1]}")
    if label_mapping:
        print(f"Target label mapping: {label_mapping}")

    return (
        X_train, X_val, X_test,
        y_train, y_val, y_test,
        None, le, class_weight
    )


## COMPILE AND FIT

In [ ]:
def objective(trial, name, task_type, 
              X_train, y_train, 
              X_val, y_val, 
              metric_name, categorical_cols = None, num_classes=None, SEED=42, device='cuda', save_dir=None, class_weight=None):
    params = load_search_space(name, trial)

    if name == "catboost":
        params["cat_features"] = categorical_cols

    model = get_model(name, params, task_type, num_classes, SEED, device, class_weight=class_weight)

    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)

    # Choose metric based on task
    metric_name = metric_name.lower()
    if metric_name == "f1":
        score = f1_score(y_val, y_pred, average='macro')  # F1 Score
        metric_name = "F1"

    elif metric_name == "accuracy":
        score = accuracy_score(y_val, y_pred)  # Accuracy
        metric_name = "Accuracy"

    elif metric_name == "mse":
        score = mean_squared_error(y_val, y_pred)  # MSE
        metric_name = "MSE"

    elif metric_name == "rmse":
        score = np.sqrt(mean_squared_error(y_val, y_pred))  # RMSE
        metric_name = "RMSE"

    elif metric_name == "auc":
        if hasattr(model, "predict_proba"):
            y_proba = model.predict_proba(X_val)
            if y_proba.shape[1] == 2:
                # Binary classification
                y_score = y_proba[:, 1]
                score = roc_auc_score(y_val, y_score)
            else:
                # Multiclass
                score = roc_auc_score(y_val, y_proba, multi_class="ovr", average="macro")
        else:
            raise ValueError("Model does not support predict_proba, required for AUC.")
        metric_name = "AUC"

    save_dir = os.path.join(save_dir, name, "optuna")
    os.makedirs(save_dir, exist_ok=True)

    with open(f"{save_dir}/optuna_trials_log.txt", "a") as f:
        f.write(f"Trial {trial.number} - VAL-{metric_name}: {score:.4f}, Params: {params}\n")
        f.write("=" * 60 + "\n")

    return score

In [ ]:
def evaluate_best_model(study, name, task_type,
                        X_train, y_train,
                        X_val, y_val,
                        X_test, y_test,
                        categorical_cols=None, num_classes=None, SEED=42, device='cuda', save_dir=None, class_weight=None):

    best_params = study.best_params
    
    if name == "catboost":
        best_params["cat_features"] = categorical_cols
        
    model = get_model(name, best_params, task_type, num_classes, SEED, device, class_weight=class_weight)

    save_path = os.path.join(save_dir, name, "best_model")
    os.makedirs(save_path, exist_ok=True)

    # Train model
    start_train = time.time()
    model.fit(X_train, y_train)
    train_time = time.time() - start_train

    log = {
        "Training Time (s)": train_time,
    }

    task_type = task_type.lower()

    def evaluate_split(X, y, split_name):
        start_pred = time.time()
        y_pred = model.predict(X)
        pred_time = time.time() - start_pred

        y_prob = model.predict_proba(X)[:, 1] if hasattr(model, "predict_proba") and task_type == "binary" else None
        result = {}
        result["Inference Time (s)"] = pred_time
        result["Accuracy"] = accuracy_score(y, y_pred)
        result["F1"] = f1_score(y, y_pred, average="macro")
        result["Recall"] = recall_score(y, y_pred, average="macro")
        result["Precision"] = precision_score(y, y_pred, average="macro" if task_type == "multiclass" else "binary")
        result["Balanced Accuracy"] = balanced_accuracy_score(y, y_pred)

        if task_type == "binary":
            result["MCC"] = matthews_corrcoef(y, y_pred)
        
        if task_type == "binary" and y_prob is not None:
            result["AUC"] = roc_auc_score(y, y_prob)
            result["Avg Precision"] = average_precision_score(y, y_prob)
            result["Log Loss"] = log_loss(y, y_prob)

        # Print results directly
        print(f"\n--- {split_name} Results ---")
        for k, v in result.items():
            print(f"{k}: {v:.4f}")

        return {f"{split_name} {k}": v for k, v in result.items()}

    if task_type == "regression":
        def evaluate_regression(X, y, split_name):
            y_pred = model.predict(X)
            result = {
                f"{split_name} MSE": mean_squared_error(y, y_pred),
                f"{split_name} RMSE": np.sqrt(mean_squared_error(y, y_pred)),
                f"{split_name} R2": r2_score(y, y_pred)
            }
            print(f"\n--- {split_name} Regression Results ---")
            for k, v in result.items():
                print(f"{k}: {v:.4f}")
            return result

        log.update(evaluate_regression(X_train, y_train, "Train"))
        log.update(evaluate_regression(X_val, y_val, "Val"))
        log.update(evaluate_regression(X_test, y_test, "Test"))

    elif task_type in ["binary", "multiclass"]:
        log.update(evaluate_split(X_train, y_train, "Train"))
        log.update(evaluate_split(X_val, y_val, "Val"))
        log.update(evaluate_split(X_test, y_test, "Test"))

    # Save model
    model_file = os.path.join(save_path, "best_model.joblib")
    joblib.dump(model, model_file)

    # Save metrics
    log_file = os.path.join(save_path, "best_model_metrics.txt")
    with open(log_file, "w") as f:
        for key, value in log.items():
            f.write(f"{key}: {value:.4f}\n")

    print("\nBest Parameters:")
    print(best_params)

    # Save best hyperparameters
    params_file = os.path.join(save_path, "best_params.json")
    with open(params_file, "w") as f:
        json.dump(best_params, f, indent=4)

    print(f"\nModel saved: {model_file}")
    print(f"Metrics saved: {log_file}")
    return log


In [ ]:
import random
import numpy as np
import torch

def set_model_seed(seed: int):
    # Python built-in RNG
    random.seed(seed)
    # NumPy RNG
    np.random.seed(seed)
    # Torch RNG
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if you use multi-GPU
    
    # For reproducibility
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

## EXPERIMENTS

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")
save_dir =  os.path.join("logs", task_type, dataset_name)

In [ ]:
# Define the metric and direction based on task_type
if task_type.lower() == 'regression':
    metric_name = "RMSE"  # or any other regression metric
    direction = "minimize"  # Lower RMSE is better
elif task_type.lower() == 'binary':
    metric_name = "AUC"  # or any other binary classification metric
    direction = "maximize"  # Higher AUC is better
elif task_type.lower() == 'multiclass':
    metric_name = "Accuracy"  # or any other multiclass classification metric
    direction = "maximize"  # Higher accuracy is better
else:
    raise ValueError(f"Unknown task_type: {task_type}")

print(metric_name, direction)

if task_type.lower() == 'regression' or task_type.lower() == 'binary':
    num_classes = None
else:
    num_classes = df.iloc[:, -1].nunique()
    print(f"Number of classes: {num_classes}")

## XGBoost

In [ ]:
save_dir

In [ ]:
X_train, X_val, X_test, y_train, y_val, y_test, categorical_cols, label_encoder, class_weight = preprocess_data(df, dataset_name=dataset_name, task_type=task_type, model_type="xgboost", seed=SEED)

In [ ]:
study = optuna.create_study(direction=direction)
study.optimize(lambda trial: objective(
    trial=trial,
    name="xgboost",
    task_type=task_type,
    num_classes=num_classes,
    X_train=X_train,
    y_train=y_train,
    X_val=X_val,
    y_val=y_val,
    metric_name=metric_name,
    SEED=SEED,
    device=device,
    save_dir=save_dir,
    class_weight=class_weight,
), n_trials=100)

# Print best result summary
best_trial = study.best_trial
print(f"\nBest Trial: {best_trial.number}")
print(f"  Score: {best_trial.value:.4f}")
print("  Best Hyperparameters:")
for k, v in best_trial.params.items():
    print(f"    {k}: {v}")

In [ ]:
from numbers import Number

# seeds & aggregation
model_seeds = [0, 1, 2, 3, 4]
numeric_keys = None
per_seed_metrics = []

# Where to save the summary file (the same folder you used before)
summary_dir = os.path.join(save_dir, f"xgboost/best_model")
os.makedirs(summary_dir, exist_ok=True)
out_file = os.path.join(summary_dir, "best_results_mean.txt")

for s in model_seeds:
    set_model_seed(s)  # your util to set np/torch/python seeds if needed
    metrics = evaluate_best_model(
        study,                 # Optuna study with .best_params
        "xgboost",                  # "xgboost" / "lightgbm" / "catboost" / etc.
        task_type,
        X_train, y_train,
        X_val, y_val,
        X_test, y_test,
        categorical_cols=categorical_cols,
        num_classes=num_classes,
        SEED=s,                # <<< pass the seed into the model
        device=device,
        save_dir=save_dir,
        class_weight=class_weight
    )
    if not isinstance(metrics, dict):
        raise TypeError(f"evaluate_best_model must return dict, got {type(metrics)}")

    if numeric_keys is None:
        numeric_keys = [k for k, v in metrics.items() if isinstance(v, (Number, np.floating, np.integer))]
    per_seed_metrics.append(metrics)

    # brief print
    brief = ", ".join(f"{k}={float(metrics[k]):.6f}" for k in numeric_keys[:6])
    print(f"Seed {s}: {brief}")

# Aggregate mean/std
aggregates = {}
for k in numeric_keys:
    vals = [float(m[k]) for m in per_seed_metrics]
    mean_k = float(np.mean(vals))
    std_k  = float(np.std(vals, ddof=1)) if len(vals) > 1 else 0.0
    aggregates[k] = {"mean": mean_k, "std": std_k}

# Save YAML-like txt (same style as your example)
with open(out_file, "w", encoding="utf-8") as f:
    f.write("# Best trial re-evaluation across model seeds\n")
    # If you really want to include patch_size (only exists for ViT/CNN), guard it:
    if hasattr(study, "best_params") and "patch_size" in study.best_params:
        f.write(f"patch_size: {study.best_params['patch_size']}\n")
    f.write(f"seeds: {model_seeds}\n")
    f.write("per_seed_metrics:\n")
    for s, m in zip(model_seeds, per_seed_metrics):
        f.write(f"  - seed: {s}\n")
        for k in numeric_keys:
            f.write(f"      {k}: {float(m[k]):.6f}\n")
    f.write("aggregates:\n")
    for k, mm in aggregates.items():
        f.write(f"  {k}:\n")
        f.write(f"    mean: {mm['mean']:.6f}\n")
        f.write(f"    std: {mm['std']:.6f}\n")

# Console summary (pick a sensible key)
pref_key = None
if task_type.lower() == "binary":
    for cand in ["Test AUC", "Test Accuracy", "Val AUC", "Val Accuracy"]:
        if cand in aggregates: pref_key = cand; break
elif task_type.lower() == "multiclass":
    for cand in ["Test Accuracy", "Val Accuracy", "Test F1", "Val F1"]:
        if cand in aggregates: pref_key = cand; break
else:  # regression
    for cand in ["Test RMSE", "Val RMSE", "Test R2", "Val R2"]:
        if cand in aggregates: pref_key = cand; break

if pref_key:
    print(f"→ xgboost: {pref_key} = {aggregates[pref_key]['mean']:.6f} ± {aggregates[pref_key]['std']:.6f}")
print(f"Saved to: {out_file}")


## Catboost

In [ ]:
save_dir

In [ ]:
X_train, X_val, X_test, y_train, y_val, y_test, categorical_cols, label_encoder, class_weight = preprocess_data(df, dataset_name=dataset_name, task_type=task_type, model_type="catboost", seed=SEED)

In [ ]:
# Load config
with open(f"./configs/preprocess/{dataset_name}.json") as f:
    config = json.load(f)

categorical_cols = config["categorical_cols"]

In [ ]:
study = optuna.create_study(direction=direction)
study.optimize(lambda trial: objective(
    trial=trial,
    name="catboost",
    task_type=task_type,
    num_classes=num_classes,
    X_train=X_train,
    y_train=y_train,
    X_val=X_val,
    y_val=y_val,
    metric_name=metric_name,
    SEED=SEED,
    device=device,
    save_dir=save_dir,
    class_weight=class_weight,
    categorical_cols=categorical_cols
), n_trials=50)

# Print best result summary
best_trial = study.best_trial
print(f"\nBest Trial: {best_trial.number}")
print(f"  AUC Score: {best_trial.value:.4f}")
print("  Best Hyperparameters:")
for k, v in best_trial.params.items():
    print(f"    {k}: {v}")

In [ ]:
from numbers import Number

# seeds & aggregation
model_seeds = [0, 1, 2, 3, 4]
numeric_keys = None
per_seed_metrics = []

# Where to save the summary file (the same folder you used before)
summary_dir = os.path.join(save_dir, f"catboost/best_model")
os.makedirs(summary_dir, exist_ok=True)
out_file = os.path.join(summary_dir, "best_results_mean.txt")

for s in model_seeds:
    set_model_seed(s)  # your util to set np/torch/python seeds if needed
    metrics = evaluate_best_model(
        study,                 # Optuna study with .best_params
        "catboost",                  # "xgboost" / "lightgbm" / "catboost" / etc.
        task_type,
        X_train, y_train,
        X_val, y_val,
        X_test, y_test,
        categorical_cols=categorical_cols,
        num_classes=num_classes,
        SEED=s,                # <<< pass the seed into the model
        device=device,
        save_dir=save_dir,
        class_weight=class_weight
    )
    if not isinstance(metrics, dict):
        raise TypeError(f"evaluate_best_model must return dict, got {type(metrics)}")

    if numeric_keys is None:
        numeric_keys = [k for k, v in metrics.items() if isinstance(v, (Number, np.floating, np.integer))]
    per_seed_metrics.append(metrics)

    # brief print
    brief = ", ".join(f"{k}={float(metrics[k]):.6f}" for k in numeric_keys[:6])
    print(f"Seed {s}: {brief}")

# Aggregate mean/std
aggregates = {}
for k in numeric_keys:
    vals = [float(m[k]) for m in per_seed_metrics]
    mean_k = float(np.mean(vals))
    std_k  = float(np.std(vals, ddof=1)) if len(vals) > 1 else 0.0
    aggregates[k] = {"mean": mean_k, "std": std_k}

# Save YAML-like txt (same style as your example)
with open(out_file, "w", encoding="utf-8") as f:
    f.write("# Best trial re-evaluation across model seeds\n")
    # If you really want to include patch_size (only exists for ViT/CNN), guard it:
    if hasattr(study, "best_params") and "patch_size" in study.best_params:
        f.write(f"patch_size: {study.best_params['patch_size']}\n")
    f.write(f"seeds: {model_seeds}\n")
    f.write("per_seed_metrics:\n")
    for s, m in zip(model_seeds, per_seed_metrics):
        f.write(f"  - seed: {s}\n")
        for k in numeric_keys:
            f.write(f"      {k}: {float(m[k]):.6f}\n")
    f.write("aggregates:\n")
    for k, mm in aggregates.items():
        f.write(f"  {k}:\n")
        f.write(f"    mean: {mm['mean']:.6f}\n")
        f.write(f"    std: {mm['std']:.6f}\n")

# Console summary (pick a sensible key)
pref_key = None
if task_type.lower() == "binary":
    for cand in ["Test AUC", "Test Accuracy", "Val AUC", "Val Accuracy"]:
        if cand in aggregates: pref_key = cand; break
elif task_type.lower() == "multiclass":
    for cand in ["Test Accuracy", "Val Accuracy", "Test F1", "Val F1"]:
        if cand in aggregates: pref_key = cand; break
else:  # regression
    for cand in ["Test RMSE", "Val RMSE", "Test R2", "Val R2"]:
        if cand in aggregates: pref_key = cand; break

if pref_key:
    print(f"→ catboost: {pref_key} = {aggregates[pref_key]['mean']:.6f} ± {aggregates[pref_key]['std']:.6f}")
print(f"Saved to: {out_file}")


## LightGBM

In [ ]:
save_dir

In [ ]:
X_train, X_val, X_test, y_train, y_val, y_test, categorical_cols, label_encoder, class_weight = preprocess_data(df, dataset_name=dataset_name, task_type=task_type, model_type="lightgbm", seed=SEED)

In [ ]:
# Load config
with open(f"./configs/preprocess/{dataset_name}.json") as f:
    config = json.load(f)

categorical_cols = config["categorical_cols"]

In [ ]:
study = optuna.create_study(direction=direction)
study.optimize(lambda trial: objective(
    trial=trial,
    name="lightgbm",
    task_type=task_type,
    num_classes=num_classes,
    X_train=X_train,
    y_train=y_train,
    X_val=X_val,
    y_val=y_val,
    metric_name=metric_name,
    SEED=SEED,
    device=device,
    save_dir=save_dir,
    class_weight=class_weight,
    categorical_cols=categorical_cols
), n_trials=50)

# Print best result summary
best_trial = study.best_trial
print(f"\nBest Trial: {best_trial.number}")
print(f"  AUC Score: {best_trial.value:.4f}")
print("  Best Hyperparameters:")
for k, v in best_trial.params.items():
    print(f"    {k}: {v}")

In [ ]:
from numbers import Number

# seeds & aggregation
model_seeds = [0, 1, 2, 3, 4]
numeric_keys = None
per_seed_metrics = []

# Where to save the summary file (the same folder you used before)
summary_dir = os.path.join(save_dir, f"lightgbm/best_model")
os.makedirs(summary_dir, exist_ok=True)
out_file = os.path.join(summary_dir, "best_results_mean.txt")

for s in model_seeds:
    set_model_seed(s)  # your util to set np/torch/python seeds if needed
    metrics = evaluate_best_model(
        study,                 # Optuna study with .best_params
        "lightgbm",                  # "xgboost" / "lightgbm" / "catboost" / etc.
        task_type,
        X_train, y_train,
        X_val, y_val,
        X_test, y_test,
        categorical_cols=categorical_cols,
        num_classes=num_classes,
        SEED=s,                # <<< pass the seed into the model
        device=device,
        save_dir=save_dir,
        class_weight=class_weight
    )
    if not isinstance(metrics, dict):
        raise TypeError(f"evaluate_best_model must return dict, got {type(metrics)}")

    if numeric_keys is None:
        numeric_keys = [k for k, v in metrics.items() if isinstance(v, (Number, np.floating, np.integer))]
    per_seed_metrics.append(metrics)

    # brief print
    brief = ", ".join(f"{k}={float(metrics[k]):.6f}" for k in numeric_keys[:6])
    print(f"Seed {s}: {brief}")

# Aggregate mean/std
aggregates = {}
for k in numeric_keys:
    vals = [float(m[k]) for m in per_seed_metrics]
    mean_k = float(np.mean(vals))
    std_k  = float(np.std(vals, ddof=1)) if len(vals) > 1 else 0.0
    aggregates[k] = {"mean": mean_k, "std": std_k}

# Save YAML-like txt (same style as your example)
with open(out_file, "w", encoding="utf-8") as f:
    f.write("# Best trial re-evaluation across model seeds\n")
    # If you really want to include patch_size (only exists for ViT/CNN), guard it:
    if hasattr(study, "best_params") and "patch_size" in study.best_params:
        f.write(f"patch_size: {study.best_params['patch_size']}\n")
    f.write(f"seeds: {model_seeds}\n")
    f.write("per_seed_metrics:\n")
    for s, m in zip(model_seeds, per_seed_metrics):
        f.write(f"  - seed: {s}\n")
        for k in numeric_keys:
            f.write(f"      {k}: {float(m[k]):.6f}\n")
    f.write("aggregates:\n")
    for k, mm in aggregates.items():
        f.write(f"  {k}:\n")
        f.write(f"    mean: {mm['mean']:.6f}\n")
        f.write(f"    std: {mm['std']:.6f}\n")

# Console summary (pick a sensible key)
pref_key = None
if task_type.lower() == "binary":
    for cand in ["Test AUC", "Test Accuracy", "Val AUC", "Val Accuracy"]:
        if cand in aggregates: pref_key = cand; break
elif task_type.lower() == "multiclass":
    for cand in ["Test Accuracy", "Val Accuracy", "Test F1", "Val F1"]:
        if cand in aggregates: pref_key = cand; break
else:  # regression
    for cand in ["Test RMSE", "Val RMSE", "Test R2", "Val R2"]:
        if cand in aggregates: pref_key = cand; break

if pref_key:
    print(f"→ lightgbm: {pref_key} = {aggregates[pref_key]['mean']:.6f} ± {aggregates[pref_key]['std']:.6f}")
print(f"Saved to: {out_file}")
